# 8 Puzzle Assignment  

**Minerva Schools at KGI  
Author: Zhanchen Guo**

In [71]:
import numpy as np
from queue import PriorityQueue
from copy import deepcopy

class PuzzleNode:
    
    def __init__(self, n, state, parent=None):
        """
        n: (int) n * n size of the board
        state: (2d list) the board status
        parent: (PuzzleNode object) parent node
        """
        self.n = n
        self.raw_state = state
        self.state = np.array(state)
        self.parent = parent
        self.pos_zero = self._find_zero()
        self.depth = self._find_depth()
        
    def state_validation(self):
        if self.n < 3 or self.state.shape != (self.n, self.n) or (not np.array_equal(np.sort(self.state, axis=None), np.arange(self.n ** 2))):
            return -1
        else:
            return 0
    
    def _find_depth(self):
        return 0 if not self.parent else self.parent.depth + 1
            
    def _find_zero(self):
        position = np.where(self.state == 0)
        return (position[0][0], position[1][0]) # (y, x)
    
    def _valid_position(self, y, x):
        return True if 0 <= y < self.n and 0 <= x < self.n else False
        
    def available_moves(self):
        possible_moves = ((self.pos_zero[0], self.pos_zero[1] - 1), 
                         (self.pos_zero[0], self.pos_zero[1] + 1), 
                         (self.pos_zero[0] - 1, self.pos_zero[1]),
                         (self.pos_zero[0] + 1, self.pos_zero[1]))
        moves = []
        for y, x in possible_moves:
            if self._valid_position(y, x):
                if self.parent and self.parent.pos_zero == (y, x): # do not go back
                    continue
                moves.append((y, x))
        return moves

    def move_to(self, next_move): 
        """
        next_move: (y, x) Position of the next move
        """
        state = deepcopy(self.raw_state) #so that raw_state will not change
        # swap algorithm
        c = state[next_move[0]][next_move[1]]
        state[next_move[0]][next_move[1]] = 0
        state[self.pos_zero[0]][self.pos_zero[1]] = c
        return state
    
    def __eq__(self, state):
        return True if state.raw_state == self.raw_state else False
    
    def __lt__(self, state): #only for priority queue
        return self.depth < state.depth
        
    def __str__(self):
        return str(self.state)
        


# PuzzleNode Class

## paramters

n: n*n board  
raw_state: list version board  
state: numpy array version board  
parent: parent node  
pos_zero is the position of number 0 in the board, format (y, x)  
depth is the depth of the current node in the search tree. It is equal to steps of the search.  

## methods:

**`state_validation()`**
make sure sure the inputed state are valid and passing three validation factors:
1. n at least 3
2. shape is n * n
3. board contains all the number from 0 to n^2 - 1 with no repeat

**`available_moves()`** 
Return all the possible moves based on the current board status.  
Since there are maximum 4 moves, from top, bottom, left and right. I listed the four possible moves and then check if there are cell exits in these four position using internal method *_valid_position()*.  

**`move_to(next_move)`** 
Return the list for the state where the next move take in place. It basically just swap the 0 location and the next move location

### Internal methods

**`_find_depth()`**  
Find the depth of given node, the depth is now far the current node from its root parent. Since each node keep track of their own depth, I do not need to trace down and simply depth = parents.depth + 1. Parent has depth 0.  

**`_find_zero()`**  
Find the location of the 0.  

**`_valid_position(y, x)`**  
Check if x and y is within the boundary of 0 to n.  

### Overloads

**`__eq__`**  
This for the use of "in" operator in the solvePuzzle function. Used to compare if two state are the same. 

**`__lt__`**  
This is for preventing the error from PriorityQueue library. The library requires all data in .put() are comparable.

**`__str__`**  
To conveniently print the table


In [62]:
def misplaced_tiles(state):
    """state (list of list)
    """
    state = np.array(state)
    goal = np.arange(state.size)
    goal.shape = state.shape
    return state.size - np.sum(goal == state)

def manhattan_distance(state):
    """state (list of list)
    """
    n = len(state)
    distance = 0
    location = [None] * (n ** 2)
    target = []
    
    for i in range(n):
        for j in range(n):
            location[state[i][j]] = (i, j)
            target.append((i, j))

    for i in range(len(location)):
        curr_x, curr_y = location[i] 
        targ_x, targ_y = target[i]
        distance += abs(targ_x - curr_x) + abs(targ_y - curr_y)
    return distance

heuristics = [misplaced_tiles, manhattan_distance]

def solvePuzzle(n, state, heuristic, prnt):
    """
    n: n 
    state: (list of list)
    """
    root = PuzzleNode(n, state)
    err = root.state_validation()
    if err == -1:
        return 0, 0, -1
    
    frontier = PriorityQueue()
    frontier.put((0, root))
    opened = []
    while not frontier.empty():
        priority, state = frontier.get()
        if prnt:
            print(state)
            
        if misplaced_tiles(state.raw_state) == 0: #check if goal reached
            return state.depth, frontier.qsize(), err
        
        moves = state.available_moves()
        for move in moves:
            next_state = PuzzleNode(n, state.move_to(move), parent=state)
            if next_state in opened: #prevent repeat route
                continue
            priority = next_state.depth + heuristic(next_state.raw_state) #g(h) + f(h)
            frontier.put((priority, next_state))
            opened.append(next_state)
    return 0, 0 , -1 # case there is no solution



# Function Algorithm Explain

## misplaced_tiles

Firstly, construct a goal state which has the same format as the input state.
and then use goal == state, numpy array will compare each cell and give True of False values.
use np.sum to count all the True value, which are all the corrected placed tiles. 
Use the total numbers of tiles minus the correct tiles is the number of misplaced tiles. 

## manhattan_distance

Set a location array. The Index of this array represent the numbers in the board, number 5 in the board location (2, 3), means `location[5] == (2, 3)`. Literate through the board, grab all the location data into this array. 
Target array is the goal state equivalent of location array.  
Target array and location array is parallel array representing the goal location and current location.  
Iterate through and calculate manhattan distance

## solvePuzzle

Firstly create root node, and then put root node in priority queue. opened list is to prevent going into the same state, all the visited and frontier states will be in opened.
In while loop, firstly check if we need to print the state, secondly check if goal condition reached.  
Later figure out all the possible moves at the current node, check if the possible mode is already explored. If not, using heuristic function calculate priority and put it into both the priority queue and the opened array.
If we explore all the possibility yet still not result, we will return -1 for error, the reminder paramter is 0.

Note that the steps is the depth of the final node reached the goal state, the frontier size is the size of the priority queue. 


In [73]:
test1 = [[5,7,6],[2,4,3],[8,1,0]]
steps, frontiersize, err = solvePuzzle(3, test, heuristics[1], False)
print("Using Manhatten Distance")
print("Steps: " + str(steps))
print("Frontier Size: " + str(frontiersize))
print("Error code: " + str(err))

# should return error -1
test1 = [[5,7,6],[2,4,3],[8,1,0]]
steps, frontiersize, err = solvePuzzle(4, test, heuristics[1], False)
print("Using Manhatten Distance for erroreous input")
print("Steps: " + str(steps))
print("Frontier Size: " + str(frontiersize))
print("Error code: " + str(err))

# Note that this runs forever
# steps, frontiersize, err = solvePuzzle(3, test, heuristics[0], False)
# print("Using misplace tiles Distance")
# print("Steps: " + str(steps))
# print("Frontier Size: " + str(frontiersize))
# print("Error code: " + str(err))

Using Manhatten Distance
Steps: 28
Frontier Size: 2759
Error code: 0
Using Manhatten Distance for erroreous input
Steps: 0
Frontier Size: 0
Error code: -1


In [74]:
test2 = [[7,0,8],[4,6,1],[5,3,2]]
steps, frontiersize, err = solvePuzzle(3, test2, heuristics[1], False)
print("Using Manhatten Distance")
print("Steps: " + str(steps))
print("Frontier Size: " + str(frontiersize))
print("Error code: " + str(err))

# Note that this runs forever
# steps, frontiersize, err = solvePuzzle(3, test2, heuristics[0], False)
# print("Using misplace tiles Distance")
# print("Steps: " + str(steps))
# print("Frontier Size: " + str(frontiersize))
# print("Error code: " + str(err))

Using Manhatten Distance
Steps: 25
Frontier Size: 2046
Error code: 0


In [69]:
test3 = [[2,3,7],[1,8,0],[6,5,4]]
steps, frontiersize, err = solvePuzzle(3, test3, heuristics[1], False)
print("Using Manhatten Distance")
print("Steps: " + str(steps))
print("Frontier Size: " + str(frontiersize))
print("Error code: " + str(err))

steps, frontiersize, err = solvePuzzle(3, test3, heuristics[0], False)
print("Using misplace tiles Distance")
print("Steps: " + str(steps))
print("Frontier Size: " + str(frontiersize))
print("Error code: " + str(err))

Using Manhatten Distance
Steps: 17
Frontier Size: 109
Error code: 0
Using misplace tiles Distance
Steps: 17
Frontier Size: 558
Error code: 0


# Observation

Bsaed on the three test cases. 
All cases Manhatten distance can reach conclusion in a short time. Misplaced Tiles only reach conclusion at the last case but with much larger frontier size. The first and second test cases it does not return a conclusion in a reasonable amount of time. Therefore Manhatten distance is much better heuristic to use. 